## Load and Organize data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install contractions emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 8.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import re
import regex as re
import string
import nltk
from collections import defaultdict

from textblob import TextBlob

import emoji
import contractions
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.decomposition import TruncatedSVD

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import LinearSVC

In [ ]:
# Scaricare risorse NLTK necessarie
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# Downloading GoEmotions dataset from google Apis ..

! wget -P data/full_dataset/ https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_1.csv
! wget -P data/full_dataset/ https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_2.csv
! wget -P data/full_dataset/ https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_3.csv

--2025-01-04 14:07:37--  https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_1.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.125.207, 142.250.157.207, 142.251.8.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.125.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14174600 (14M) [application/octet-stream]
Saving to: ‘data/full_dataset/goemotions_1.csv’

goemotions_1.csv    100%[===================>]  13.52M  8.02MB/s    in 1.7s    

2025-01-04 14:07:39 (8.02 MB/s) - ‘data/full_dataset/goemotions_1.csv’ saved [14174600/14174600]

--2025-01-04 14:07:39--  https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_2.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.125.207, 142.250.157.207, 142.251.8.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.125.207|:443... connected.
HTTP request sent, awaiting r

In [ ]:
# Merging all dataframes, cleaned by the examples defined as "very unclear"

df1 = pd.read_csv('/content/data/full_dataset/goemotions_1.csv')
df2 = pd.read_csv('/content/data/full_dataset/goemotions_2.csv')
df3 = pd.read_csv('/content/data/full_dataset/goemotions_3.csv')

print(len(df1), len(df2), len(df3))

df1 = df1[df1.iloc[:, 9:].sum(axis = 1) == 1]
df2 = df2[df2.iloc[:, 9:].sum(axis = 1) == 1]
df3 = df3[df3.iloc[:, 9:].sum(axis = 1) == 1]

print(len(df1), len(df2), len(df3))

df1[df1['example_very_unclear'] == 'False']
df2[df2['example_very_unclear'] == 'False']
df3[df3['example_very_unclear'] == 'False']

print(len(df1), len(df2), len(df3))

70000 70000 71225
56984 56964 57872
56984 56964 57872


In [ ]:
df = pd.concat([df1, df2, df3], ignore_index=True)
print(len(df))

171820


In [ ]:
# Dropping unnecessary features

df = df.drop(columns = ['id', 'author', 'subreddit', 'link_id', 'parent_id', 'created_utc', 'rater_id', 'example_very_unclear'])

# Identify emotion columns

emotion_columns = [col for col in df.columns if col not in ['text']]

# Find the corresponding label for each row

df['label'] = df[emotion_columns].idxmax(axis=1)
df = df.drop(columns = emotion_columns)
df.head()

,text,label
0,That game hurt.,sadness
1,"You do right, if you don't care then fuck 'em!",neutral
2,Man I love reddit.,love
3,"[NAME] was nowhere near them, he was by the Fa...",neutral
4,Right? Considering it’s such an important docu...,gratitude


In [ ]:
# Maps useful to target the emotions considered for our purpose

map_1= {
"anger": ["anger", "annoyance", "disapproval"],
"disgust": ["disgust"],
"fear": ["fear", "nervousness"],
"joy": ["joy", "amusement", "approval", "excitement", "gratitude",  "love", "optimism", "relief", "pride", "admiration", "desire", "caring"],
"sadness": ["sadness", "disappointment", "embarrassment", "grief",  "remorse"],
"surprise": ["surprise", "realization", "confusion", "curiosity"],
'neutral': ['neutral']
}

map_2 = {
"positive": ["amusement", "excitement", "joy", "love", "desire", "optimism", "caring", "pride", "admiration", "gratitude", "relief", "approval"],
"negative": ["fear", "nervousness", "remorse", "embarrassment", "disappointment", "sadness", "grief", "disgust", "anger", "annoyance", "disapproval"],
"ambiguous": ["realization", "surprise", "curiosity", "confusion"],
'neutral': ['neutral']
}


inverse_map_1 = {emotion: category for category, emotions in map_1.items() for emotion in emotions}
inverse_map_2 = {emotion: sentiment for sentiment, emotions in map_2.items() for emotion in emotions}

df['mapped_emotion'] = df['label'].map(inverse_map_1)
df['mapped_sentiment'] = df['label'].map(inverse_map_2)

df.head()

,text,label,mapped_emotion,mapped_sentiment
0,That game hurt.,sadness,sadness,negative
1,"You do right, if you don't care then fuck 'em!",neutral,neutral,neutral
2,Man I love reddit.,love,joy,positive
3,"[NAME] was nowhere near them, he was by the Fa...",neutral,neutral,neutral
4,Right? Considering it’s such an important docu...,gratitude,joy,positive


In [ ]:
print(f"Initial length: {len(df)} \n")

# Step 1: Group the texts to check for label diversity

grouped = df.groupby("text")["mapped_emotion"].nunique().reset_index()
grouped.columns = ["text", "unique_labels"]

# Step 2: Filter the texts that appear with a single label

texts_with_consensus = grouped[grouped["unique_labels"] == 1]["text"]

# Step 3: Filter the original DataFrame to keep only unique or consensus texts

filtered_df = df[df["text"].isin(texts_with_consensus)]

# Remove any duplicates by leaving only one example per text

filtered_df = filtered_df.drop_duplicates(subset="text").reset_index(drop=True)

filtered_df

Initial length: 171820 



,text,label,mapped_emotion,mapped_sentiment
0,Man I love reddit.,love,joy,positive
1,"[NAME] was nowhere near them, he was by the Fa...",neutral,neutral,neutral
2,Right? Considering it’s such an important docu...,gratitude,joy,positive
3,that's adorable asf,amusement,joy,positive
4,Build a wall? /jk,neutral,neutral,neutral
...,...,...,...,...
23690,Everyone likes [NAME].,love,joy,positive
23691,"Hahaha, ok. I was worried there for a second....",amusement,joy,positive
23692,He called [NAME] to the Steelers and was outsp...,neutral,neutral,neutral
23693,That guy who said that it's a good idea to nev...,admiration,joy,positive


In [ ]:
# Remove documents labeled as neutral

df = filtered_df[filtered_df.mapped_emotion != 'neutral']
df

,text,label,mapped_emotion,mapped_sentiment
0,Man I love reddit.,love,joy,positive
2,Right? Considering it’s such an important docu...,gratitude,joy,positive
3,that's adorable asf,amusement,joy,positive
6,"""Seeeee! We have one of them coloureds too!""",excitement,joy,positive
8,"Lots, play store or apple store vpn. Nord is good",admiration,joy,positive
...,...,...,...,...
23689,"Thanks, [NAME]",gratitude,joy,positive
23690,Everyone likes [NAME].,love,joy,positive
23691,"Hahaha, ok. I was worried there for a second....",amusement,joy,positive
23693,That guy who said that it's a good idea to nev...,admiration,joy,positive


## Text Processing

In [ ]:
def remove_links(text):

    # Remove links that start with http, https, ftp

    text = re.sub(r'http[s]?://\S+|ftp://\S+', ' ', text)

    # Remove links that begin with www and end with common domains (.com, .org, .net, etc.)

    text = re.sub(r'www\.[a-zA-Z0-9\-]+\.(com|org|net|edu|gov|info|io|co|uk)\S*', ' ', text)

    # Remove shortened URLs (e.g., bit.ly, tinyurl, etc.)

    text = re.sub(r'\b[a-zA-Z0-9\-]+\.(ly|gl|io|me|to|co)\b\S*', ' ', text)

    # Uniform extra spaces

    text = re.sub(r'\s+', ' ', text).strip()

    return text


In [ ]:
def correct_text(text):

    # If the text is not a string, return the original text as is

    if not isinstance(text, str):
        return text

    try:
        # Remove repetitions of characters.
        # For example, this pattern matches 'aaa', 'BBBB', etc

        pattern_alpha = re.compile(r"([A-Za-z])\1{2,}", re.DOTALL)

        # Limiting all the repetition to two characters

        formatted_text = pattern_alpha.sub(r"\1\1", text)

        # Remove repetition of punctuation

        pattern_punct = re.compile(r'([.,#!$%^&*;:{}=\-_`~()+-])\1{1,}')
        formatted_text = pattern_punct.sub(r'\1', formatted_text)

        # Correcting text with TextBlob
        # text_blb = TextBlob(formatted_text)
        # text_corrected = str(text_blb.correct())

        # if "'" in formatted_text:

        text_corrected = contractions.fix(formatted_text)

        return text_corrected

    except Exception as e:

        # If something goes wrong, return the original text

        return text


In [ ]:
df["processed_text"] = df.text.copy()

# Apply demojize to convert emoji to text

df['processed_text'] = df['processed_text'].apply(lambda x: emoji.demojize(x))

# Put everything in lowercase

df['processed_text'] = df['processed_text'].str.lower()

df.head()

<ipython-input-13-0b9108a5de11>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["processed_text"] = df.text.copy()
<ipython-input-13-0b9108a5de11>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['processed_text'] = df['processed_text'].apply(lambda x: emoji.demojize(x))
<ipython-input-13-0b9108a5de11>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pan

,text,label,mapped_emotion,mapped_sentiment,processed_text
0,Man I love reddit.,love,joy,positive,man i love reddit.
2,Right? Considering it’s such an important docu...,gratitude,joy,positive,right? considering it’s such an important docu...
3,that's adorable asf,amusement,joy,positive,that's adorable asf
6,"""Seeeee! We have one of them coloureds too!""",excitement,joy,positive,"""seeeee! we have one of them coloureds too!"""
8,"Lots, play store or apple store vpn. Nord is good",admiration,joy,positive,"lots, play store or apple store vpn. nord is good"


In [ ]:
df['processed_text'] = df['processed_text'].apply(lambda x: remove_links(x))

df['processed_text'] = df['processed_text'].apply(lambda x: correct_text(x))

df.head()

<ipython-input-14-99672b63675a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['processed_text'] = df['processed_text'].apply(lambda x: remove_links(x))
<ipython-input-14-99672b63675a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['processed_text'] = df['processed_text'].apply(lambda x: correct_text(x))


,text,label,mapped_emotion,mapped_sentiment,processed_text
0,Man I love reddit.,love,joy,positive,man i love reddit.
2,Right? Considering it’s such an important docu...,gratitude,joy,positive,right? considering it is such an important doc...
3,that's adorable asf,amusement,joy,positive,that is adorable asf
6,"""Seeeee! We have one of them coloureds too!""",excitement,joy,positive,"""see! we have one of them coloureds too!"""
8,"Lots, play store or apple store vpn. Nord is good",admiration,joy,positive,"lots, play store or apple store vpn. nord is good"


In [ ]:
slash_pattern = r"(\/[a-zA-Z]+)"

# Extract all occurrences of the pattern “/string”

matched_slashes = df.processed_text.str.extractall(slash_pattern)[0]

# Get a unique list of matched values

unique_slashes = matched_slashes.unique()

# Print the result

print("Lista unica dei pattern /stringa:")
unique_slashes.sort()
print(unique_slashes)

Lista unica dei pattern /stringa:
['/abandoned' '/abused' '/advice' '/after' '/again' '/alcohol' '/all'
 '/am' '/anxiety' '/argentinian' '/artifact' '/askreddit' '/atheist'
 '/athletic' '/atmosphere' '/austin' '/awful' '/b' '/babyelephantgifs'
 '/bdsm' '/beetlejuicing' '/begging' '/bengals' '/bestof' '/bike'
 '/blending' '/block' '/bodysuit' '/boneappletea' '/boy' '/bsv' '/bus'
 '/c' '/canada' '/changemyview' '/chat' '/childfree' '/choosingbeggars'
 '/christiangreyisdraco' '/clemson' '/closet' '/closeteverything' '/coach'
 '/coffeewhore' '/comedy' '/comedycemetery' '/comedyheaven' '/condition'
 '/conspiracy' '/consuming' '/contraception' '/could' '/county'
 '/crazyideas' '/cumosaurusgaysex' '/cypress' '/d' '/dances' '/dating'
 '/depictions' '/diazepam' '/dig' '/diner' '/dispensary' '/does' '/dogs'
 '/doing' '/dontdeadopeninside' '/drafting' '/drinking' '/drugs' '/edgy'
 '/edited' '/editing' '/eightroundsrapid' '/encinitas' '/entwives' '/etc'
 '/eu' '/events' '/exmormon' '/fandom' '/fee

In [ ]:
# Substitution dictionary

slang_dict = {
    "/s": "sarcasm",
    "/ss" : "heavy sarcasm",
    "/heavysarcasm": "heavy sarcasm",
    "/jk": "joke",
    "/rant": "frustration",
    "/whoosh": "sarcasm not understood",
    "/woosh":  "sarcasm not understood",
    "/woooosh":  "sarcasm not understood",
    "/wooooshed":   "sarcasm not understood",
    "/goofy": "weird",
    "/boneappletea" : "spelling error",
    "/murderedbywords": "clever insult",
    "/maliciouscompliance" : "passive aggressive obedience",
    "/choosingbeggars": "entitled requests",
    "/sadcringe": "sad cringe",
    "/beetlejuicing": "perfect username coincidence",
    "/comedycemetery": "comedy cemetery",
    "/iamverysmart": "i am very smart",
    "/comedyheaven": "comedy heaven",
    "/mildlyinteresting": "mildly interesting",
    "/nicegirls": "nice girls",
    "/prorevenge": "pro revenge",
    "/religious fruitcake": "cringe religion",
    "/twobestfriendwsplay": "best friends play",

    #######

    "wtf": "disbelief",
    "omg": "surprise",
    "lol": "laughter",
    "lmao": "intense laughter",
    "rofl": "intense laughter",
    "ftfy": "sarcasm",
    "tl;dr": "too long don't read",
    "imo": "in my opinion",
    "imho": "in my humbe opinion",
    "nsfw": "not safe for work",
    "nsfl": "not safe for life",
    "gg": "well done",
    "idk": "i don't know",
    "idc": "i don't care",
    "jk": "just kidding",
    "f": "pay respect",
    "yolo": "you only live once",
    "spam": "irrelevant",
    "noob": "scarce beginner",
    "troll": "provoker",
    "mfw": "my face when",
    "mrw": "my reaction when"
}

emoticon_dict = {
    ":)": "happy",
    ":-)": "happy",
    ":))": "very happy",
    ":D": "grinning",
    ":-D": "grinning",
    ";)": "winking",
    ";-)": "winking",
    ":P": "playful",
    ":-P": "playful",
    ";P": "playful",
    ";-P": "playful",
    ":'(": "crying",
    ":-(": "sad",
    ":(": "sad",
    ":-|": "neutral",
    ":|": "neutral",
    ">:(": "angry",
    ":-O": "surprised",
    ":O": "surprised",
    "xD": "laughter",
    "XD": "laughter",
    ":-/": "confused",
    ":/": "confused",
    ":3": "cute",
    ":-*": "kiss",
    ":*": "kiss",
    "<3": "love",
    ":')": "happy tears"
}

In [ ]:
# Function to replace terms with corresponding values

def replace_slang(text, slang_dict):

    # Construct a regex that combines all the keys in the dictionary

    pattern = re.compile(r'\b(' + '|'.join(re.escape(key) for key in slang_dict.keys()) + r')\b', flags=re.IGNORECASE)

    # Replace each match with the dictionary value

    return pattern.sub(lambda match: slang_dict[match.group(0).lower()], text)

# Function to replace stylized emoji

def replace_emoticons(text, emoticon_dict):

    # Build regex to capture all key emoticons

    pattern = re.compile(r'(' + '|'.join(re.escape(key) for key in emoticon_dict.keys()) + r')')

    # Replace emoticons with keywords

    return pattern.sub(lambda match: emoticon_dict[match.group(0)], text)

In [ ]:
df['processed_text'] = df['processed_text'].apply(lambda x: replace_slang(x, slang_dict))
df['processed_text'] = df['processed_text'].apply(lambda x: replace_emoticons(x, emoticon_dict))

<ipython-input-18-98524033e57a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['processed_text'] = df['processed_text'].apply(lambda x: replace_slang(x, slang_dict))
<ipython-input-18-98524033e57a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['processed_text'] = df['processed_text'].apply(lambda x: replace_emoticons(x, emoticon_dict))


In [ ]:
custom_stopwords = set([
    "a", "an", "the", "and", "or", "as", "of", "at", "by", "for", "in", "on",
    "to", "from", "about", "around", "through", "into", "across", "it", "its",
    "that", "this", "these", "those", "he",  "she", "they", "we", "us", "him",
    "her", "them", "then", "there", "here", "now", "be", "being", "been", "do",
    "does", "did", "such", "other", "another"
])

In [ ]:
# Load EmoLex

emolex = pd.read_csv("/content/drive/MyDrive/TEXT MINING/NRC-emotion-lexicon-wordlevel-alphabetized-v0.92.txt", sep="\t", header=None, names=["word", "emotion", "association"])
emolex = emolex[emolex["association"] == 1].drop("association", axis=1)

# Build an emotional dictionary

emotion_dict = defaultdict(list)
for _, row in emolex.iterrows():
    emotion_dict[row["word"]].append(row["emotion"])

# Function to get the emotions of a word

def get_emotions(word):
    return emotion_dict.get(word, [])

get_emotions("happy")

['anticipation', 'joy', 'positive', 'trust']

In [ ]:
# Function to remove punctuation

def remove_punctuation(text, punctuation=None):
    if punctuation is None:
        punctuation = string.punctuation
    return text.translate(str.maketrans(punctuation, ' ' * len(punctuation)))

# Function to remove extra spaces

def remove_extra_whitespace(text):
    return " ".join(text.split())

# Main preprocessing function

def preprocess_texts(texts):
    preprocessed_texts = []
    lemmatizer = WordNetLemmatizer()

    for text in texts:

        text= contractions.fix(text)
        # 1. Removal of punctuation and special characters.
        text = remove_punctuation(text)

        # 2. Removing multiple and excessive spaces.
        text = remove_extra_whitespace(text)

        # 3. tokenization.
        tokens = word_tokenize(text)

        # 4. Lemmatization.
        tokens = [word if get_emotions(word) else lemmatizer.lemmatize(word) for word in tokens]

        # 5. Removing numbers.
        tokens = [word for word in tokens if not word.isdigit()]

        # 6. Removing words composed of only one letter.
        tokens = [word for word in tokens if len(word) > 1]

        # 7. custom stopwords
        tokens = [word for word in tokens if word not in custom_stopwords]


        # 8. Preprocessed text reconstruction.
        preprocessed_texts.append(" ".join(tokens))

    return preprocessed_texts

In [ ]:
df['processed_text'] = preprocess_texts(df['processed_text'])
df.head()

<ipython-input-24-1e215ed4d6de>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['processed_text'] = preprocess_texts(df['processed_text'])
<ipython-input-24-1e215ed4d6de>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['document_tokens'] = df['processed_text'].apply(lambda x: x.split())


,text,label,mapped_emotion,mapped_sentiment,processed_text,document_tokens
0,Man I love reddit.,love,joy,positive,man love reddit,"[man, love, reddit]"
2,Right? Considering it’s such an important docu...,gratitude,joy,positive,right considering is important document should...,"[right, considering, is, important, document, ..."
3,that's adorable asf,amusement,joy,positive,is adorable asf,"[is, adorable, asf]"
6,"""Seeeee! We have one of them coloureds too!""",excitement,joy,positive,see have one coloureds too,"[see, have, one, coloureds, too]"
8,"Lots, play store or apple store vpn. Nord is good",admiration,joy,positive,lot play store apple store vpn nord is good,"[lot, play, store, apple, store, vpn, nord, is..."


In [ ]:
# Save the processed Dataset

df.to_csv('/content/drive/MyDrive/TEXT MINING/Data/processed_dataset.csv', index=False)